link to the dataset ---------------->>>>>  [link text](https://www.kaggle.com/rashikrahmanpritom/heart-attack-analysis-prediction-dataset)

# *Defining SVM class 

In [ ]:
import numpy as np
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
import math
class SVM:
  def __init__(self):
    self.weights=None
    self.b=None
    self.a=None
    self.sv=None
    self.sv_y=None
  
  def fit(self,X,Y):
    m,n = X.shape
    y = Y.reshape(-1,1) * 1.
    X_dash = y * X
    H = np.dot(X_dash , X_dash.T) * 1.
    P = cvxopt_matrix(H)
    q = cvxopt_matrix(-np.ones((m, 1)))
    G = cvxopt_matrix(-np.eye(m))
    h = cvxopt_matrix(np.zeros(m))
    A = cvxopt_matrix(y.reshape(1, -1))
    b = cvxopt_matrix(np.zeros(1))

    # changing default parameters
    
    cvxopt_solvers.options['show_progress'] = False
    cvxopt_solvers.options['abstol'] = 1e-10
    cvxopt_solvers.options['reltol'] = 1e-10
    cvxopt_solvers.options['feastol'] = 1e-10
    sol = cvxopt_solvers.qp(P, q, G, h, A, b)
    alphas = np.array(sol['x'])
    self.weights = np.zeros(n)
    self.weights = ((y * alphas).T @ X).reshape(-1,1)
    
    
    S = (alphas > 1e-4).flatten()
    self.b = y[S] - np.dot(X[S], self.weights)
    a = np.ravel(sol['x'])
    sv = a > 1e-5
    self.a = a[sv]
    self.sv = X[sv]
    self.sv_y = y[sv]
    self.b=self.b[0]
    val=0
    for j in range(self.weights.shape[0]):
      val+=self.weights[j]*self.weights[j];
    val=math.sqrt(val)
    margin=((np.dot(self.sv[0], self.weights) + self.b))/val
    print("---------------------------------------------------------------------------SOLUTION-------------------------------------------------------")
    print("margin value is %d" % (abs(margin)))
    print("----------------------------------------------------------------------------------------------------------------------------------")
    print("%d support vectors out of %d points" % (len(self.a), m))
    print("----------------------------------------------------------------------------------------------------------------------------------")
    print("The weight parameters are as follows---------------------------------")
    print(self.weights)
    print("----------------------------------------------------------------------------------------------------------------------------------")
    print("The bias parameter is as follow---------------------------------")
    print(self.b)
    print("----------------------------------------------------------------------------------------------------------------------------------")
    print("Support Vectors are as follows:-----------------------------------------------------")
    print(self.sv)
  
  def predict(self,X):
    return np.sign(np.dot(X, self.weights) + self.b)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [ ]:
df = pd.read_csv('heart.csv')
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [ ]:
x = df.iloc[:, 0:-1]
y = df.iloc[:, -1:]
x.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2


In [ ]:
x = x[x.columns[0:-1]].values

y=y.replace(0,-1)
y.head()


,output
0,1
1,1
2,1
3,1
4,1


In [ ]:
y.tail()

,output
298,-1
299,-1
300,-1
301,-1
302,-1


In [ ]:
y = y[y.columns[0:]].values

# converting the dataset to linearly seperable one using a logistic regression 

In [ ]:
from sklearn.linear_model import LogisticRegression
fun=LogisticRegression()
fun.fit(x, y)
y = fun.predict(x)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
from sklearn.metrics import accuracy_score
y_pred2 = fun.predict(x)
accuracy_score(y, y_pred2)

1.0

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=.2)
train_x

array([[44. ,  1. ,  1. , ...,  0. ,  2. ,  0. ],
       [66. ,  1. ,  0. , ...,  0.1,  2. ,  1. ],
       [41. ,  0. ,  2. , ...,  0. ,  2. ,  0. ],
       ...,
       [49. ,  1. ,  2. , ...,  2. ,  1. ,  3. ],
       [43. ,  1. ,  0. , ...,  0. ,  2. ,  0. ],
       [52. ,  1. ,  0. , ...,  1. ,  2. ,  2. ]])

In [ ]:
train_y

**Using hard SVM classifier**

In [ ]:
svm=SVM()
svm.fit(train_x,train_y)

---------------------------------------------------------------------------SOLUTION-------------------------------------------------------
margin value is 0
----------------------------------------------------------------------------------------------------------------------------------
13 support vectors out of 242 points
----------------------------------------------------------------------------------------------------------------------------------
The weight parameters are as follows---------------------------------
[[-0.02442885]
 [-7.73311977]
 [ 4.17466728]
 [-0.01847448]
 [-0.0418719 ]
 [-0.28254742]
 [ 0.9257799 ]
 [ 0.20520847]
 [-4.02771374]
 [-3.55047741]
 [-0.05191405]
 [-3.55913553]]
----------------------------------------------------------------------------------------------------------------------------------
The bias parameter is as follow---------------------------------
[-6.30669972]
-----------------------------------------------------------------------------------

accuracy on test data

In [ ]:
pred_y=svm .predict(test_x)
accuracy_score(test_y, pred_y)

0.9672131147540983

accuracy on training data

In [ ]:
pred_train_y=svm.predict(train_x)
accuracy_score(train_y, pred_train_y)

1.0